In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
train_dataset = pd.read_csv('data/train.csv')

### Data preprocessing & Feature Engineering
Its always a good practice to go through the following checklists to preprocess our dataset
- Split the dataset into X (attributes) and y (label)
- Check if contains missing values
- Check if continuous attribute(s) contain any outliers
- Some misc properties of the dataset

In [3]:
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


From the info object information, we can see that indeed the dataset contains missing values at the following column(s):
- Age
- Cabin

We will:
- apply **Imputer** with mean strategy to handle missing Age
- drop Cabin column since it is intuitively not useful in our analysis (as well as modeling)

In [5]:
from collections import Counter

most_common_embark = Counter(train_dataset['Embarked'].tolist()).most_common(1)[0][0]

In [6]:
most_common_embark

'S'

In [7]:
from collections import Counter

train_dataset['Embarked'].fillna(most_common_embark, inplace=True)

X = train_dataset[[
    'Sex',
    'Pclass', 
    'Age', 
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
]].values
y = train_dataset[['Survived']].values

In [8]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train_imputer_col = imputer.fit_transform(X[:, 2:3])
X[:, 2:3] = train_imputer_col

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

sex_encoder = LabelEncoder()
embarked_encoder = LabelEncoder()

X[:, 0] = sex_encoder.fit_transform(X[:, 0])
X[:, 6] = embarked_encoder.fit_transform(X[:, 6])
onehotencoder = OneHotEncoder(categorical_features = [0, 6])
X = onehotencoder.fit_transform(X).toarray()

# # Avoid dummy variables trap
X = X[:, 1:]

/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [10]:
# Prior to feature scaling
print(X[0])

# Apply feature scaling (a must if using neural network)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

[ 1.    0.    0.    1.    3.   22.    1.    0.    7.25]


In [11]:
# After feature scaling
X[0]

array([ 0.73769513, -0.48204268, -0.30756234,  0.61583843,  0.82737724,
       -0.5924806 ,  0.43279337, -0.47367361, -0.50244517])

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

### Training model with Artificial Neural Network (ANN)
Summary:<br>
Accuracy: ~84%

In [13]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [14]:
ann = Sequential()
ann.add(Dense(units=8, input_shape=(9,), activation='relu', kernel_initializer='uniform'))
ann.add(Dense(units=16, activation='relu', kernel_initializer='uniform'))
ann.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
ann.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 80        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [16]:
ann.fit(X_train, y_train, batch_size=10, epochs=100, validation_data=(X_test, y_test))

Train on 712 samples, validate on 179 samples
Epoch 1/100
712/712 [==============================] - 1s 874us/step - loss: 0.6864 - acc: 0.6138 - val_loss: 0.6687 - val_acc: 0.6201
Epoch 2/100
712/712 [==============================] - 0s 232us/step - loss: 0.6225 - acc: 0.6601 - val_loss: 0.5591 - val_acc: 0.7486
Epoch 3/100
712/712 [==============================] - 0s 238us/step - loss: 0.5286 - acc: 0.7683 - val_loss: 0.4870 - val_acc: 0.8156
Epoch 4/100
712/712 [==============================] - 0s 332us/step - loss: 0.4844 - acc: 0.7992 - val_loss: 0.4550 - val_acc: 0.8101
Epoch 5/100
712/712 [==============================] - 0s 241us/step - loss: 0.4630 - acc: 0.7879 - val_loss: 0.4397 - val_acc: 0.8101
Epoch 6/100
712/712 [==============================] - 0s 229us/step - loss: 0.4532 - acc: 0.7893 - val_loss: 0.4329 - val_acc: 0.8212
Epoch 7/100
712/712 [==============================] - 0s 253us/step - loss: 0.4451 - acc: 0.7893 - val_loss: 0.4288 - val_acc: 0.8324
Epoch 8/1

Epoch 61/100
712/712 [==============================] - 0s 277us/step - loss: 0.3883 - acc: 0.8371 - val_loss: 0.4153 - val_acc: 0.8436
Epoch 62/100
712/712 [==============================] - 0s 235us/step - loss: 0.3889 - acc: 0.8357 - val_loss: 0.4139 - val_acc: 0.8492
Epoch 63/100
712/712 [==============================] - 0s 231us/step - loss: 0.3880 - acc: 0.8399 - val_loss: 0.4145 - val_acc: 0.8492
Epoch 64/100
712/712 [==============================] - 0s 225us/step - loss: 0.3877 - acc: 0.8399 - val_loss: 0.4164 - val_acc: 0.8436
Epoch 65/100
712/712 [==============================] - 0s 275us/step - loss: 0.3890 - acc: 0.8385 - val_loss: 0.4159 - val_acc: 0.8436
Epoch 66/100
712/712 [==============================] - 0s 309us/step - loss: 0.3876 - acc: 0.8399 - val_loss: 0.4165 - val_acc: 0.8492
Epoch 67/100
712/712 [==============================] - 0s 291us/step - loss: 0.3889 - acc: 0.8371 - val_loss: 0.4164 - val_acc: 0.8492
Epoch 68/100
712/712 [==========================

In [17]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(cm)
print(report)

[[103   8]
 [ 18  50]]
              precision    recall  f1-score   support

           0       0.85      0.93      0.89       111
           1       0.86      0.74      0.79        68

   micro avg       0.85      0.85      0.85       179
   macro avg       0.86      0.83      0.84       179
weighted avg       0.86      0.85      0.85       179



### Train model with random forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [20]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(cm)
print(report)

[[100  11]
 [ 19  49]]
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       111
           1       0.82      0.72      0.77        68

   micro avg       0.83      0.83      0.83       179
   macro avg       0.83      0.81      0.82       179
weighted avg       0.83      0.83      0.83       179



### Classify the testing set for submission to Kaggle
ANN appear to perform better, hence we will use it to classify the survival for the testing set